In [2]:
from pathlib import Path
from brightway2 import *

from obd import agg_floats, custom_aggregate, import_impact_categories_db, prepare_and_write_new_db, read_and_prepare_obd_data

from utils import load_config, download_csv_file

In [ ]:
# setup brigthway stuff
projects.set_current("test1")
bw2setup()
biosphere_db =  Database("biosphere3")

In [ ]:
# Be aware that DATA-DIR is the directory where the Ökobaudat csv file is contained
# use backslash for windows ie.: Path(r'c:\my\folder with blanks\import_obd').resolve()
data_dir = Path(r'.').resolve()
print(f'data dir is: {data_dir}')

In [ ]:
# Load the configuration
config = load_config(data_dir)
#print(f'config is: {config}')

In [ ]:
# check and download oekobaudat if configured
should_dl = config['should_download']
print(f'Will download current oekobaudat: {should_dl}')
if should_dl:
    download_csv_file(config)

In [ ]:
#prepare the dataframe containing all relevant data from the oekobaudat file
obd_df = read_and_prepare_obd_data(config, data_dir)

In [ ]:
# import a database which includes biosphere proxies, ie. for each configured oekobaudat
# impact category there is one or more link to a biosphere entry which represents the respective
# obd impact category. 
# Eg. global warming potential is represented by a certain amount of co2 emission.
import_impact_categories_db(config, biosphere_db)
#import_impact_categories_meths(config)

In [ ]:
# aggregate floats by product_id
# this generates products which impact categories have the sum of all modules 
# and imports it as db to brightway, the db name is "agg_products"
all_modules_aggregated_dict = obd_df.groupby(
    [config["product_id_label"]], group_keys=False).agg(agg_floats, config["ic_names"])
# print(grouped_df)
all_modules_aggregated_dict = all_modules_aggregated_dict.reset_index()
prepare_and_write_new_db(config, "agg_products",
                         all_modules_aggregated_dict, config["product_id_label"], config["ic_names"])

In [ ]:
# make new aggregation columns: aggregate modules A1 to A5, 
# remove original columns and keep only new column: 'A1-A5', the new dataframe is called agg_A
agg_A = custom_aggregate(config, obd_df, ['A1-A3', 'A4', 'A5'], True)

# make a new column called: 'pml' and write into it the product id (config["product_id_label"]) 
# plus the module. This enables importing the modules of products as own activities 
# which allows doing lca over certain modules only.
mod_column_label = 'pml'
agg_A[mod_column_label] = agg_A.apply(
    lambda row: row[config["product_id_label"]] + '#' + row['Modul'], axis=1)

# write this new dataframe: agg_A to a brightway db called mod_products
prepare_and_write_new_db(config, "mod_products", agg_A, mod_column_label, config["ic_names"])